In [ ]:
#@title Regulatory Gene Network Inference
DATASET = 'Yeastract' #@param["Yeastract","DREAM5"]

#@markdown ### Training Parameters
Epochs    =  50#@param
BatchSize =  64 #@param
Classification = 'Connection-Only' #@param ['Connection-Only','Connection_w_Regulation']
UseDefault = False #@param {type:"boolean"}

GetROCcurve  = True #@param {type:"boolean"}
GetPRcurve   = True #@param {type:"boolean"}
GetMCC       = True #@param {type:"boolean"}
SaveModel = True #@param {type:"boolean"}
SaveDir   =  '/content/trial/'#@param {type:"string"}

params = {'train_p':{'epochs':Epochs,'batchsize':BatchSize,'n_classes':4 if 'regulation' in Classification.lower() else 2},
          'log_p':[GetROCcurve,GetPRcurve,GetMCC,SaveModel,SaveDir],
          'dataset':DATASET}

if UseDefault:
  params = {'train_p':{'epochs':300,'batchsize':64 if 'yeastract' in DATASET.lower() else 8,'n_classes':4 },
          'log_p':[True,True,True,True,'/content/default_trained'],
          'dataset':DATASET}

#@markdown ---
#@markdown ### Start Training
Train = True #@param {type:"boolean"}

if Train:
  start_training(params)

2
5388
674
674
Epoch 1/50
84/84 [==============================] - 7s 46ms/step - loss: 0.9603 - Accuracy: 0.5757 - auc_5: 0.6000 - val_loss: 0.7587 - val_Accuracy: 0.6706 - val_auc_5: 0.7177
Epoch 2/50
84/84 [==============================] - 3s 40ms/step - loss: 0.7688 - Accuracy: 0.6360 - auc_5: 0.6840 - val_loss: 0.6337 - val_Accuracy: 0.6884 - val_auc_5: 0.7468
Epoch 3/50
84/84 [==============================] - 6s 71ms/step - loss: 0.6780 - Accuracy: 0.6784 - auc_5: 0.7283 - val_loss: 0.6540 - val_Accuracy: 0.7092 - val_auc_5: 0.7698
Epoch 4/50
84/84 [==============================] - 3s 40ms/step - loss: 0.6156 - Accuracy: 0.7039 - auc_5: 0.7636 - val_loss: 0.5632 - val_Accuracy: 0.7478 - val_auc_5: 0.7947
Epoch 5/50
84/84 [==============================] - 3s 40ms/step - loss: 0.5885 - Accuracy: 0.7147 - auc_5: 0.7781 - val_loss: 0.5763 - val_Accuracy: 0.7255 - val_auc_5: 0.7985
Epoch 6/50
84/84 [==============================] - 6s 71ms/step - loss: 0.5554 - Accuracy: 0.7308 -

KeyboardInterrupt: ignored

# Run First

In [ ]:
!git clone https://github.com/AhmedFakhry47/Regulatory-Gene-Network-Inferance-from-gene-exepression-data

from sklearn.metrics import roc_curve,roc_auc_score,auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from matplotlib import pyplot as plt
from keras.callbacks import Callback
from tensorflow import keras
from itertools import cycle
from scipy import interp
import tensorflow as tf
import numpy as np
import subprocess
import pickle
import json
import sys
import os

Cloning into 'Regulatory-Gene-Network-Inferance-from-gene-exepression-data'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 782 (delta 28), reused 78 (delta 28), pack-reused 704
Receiving objects: 100% (782/782), 262.63 MiB | 20.98 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
def start_training(params):
  '''
  This function takes:
  1- Training parameters such as batch size, # of epochs, etc.
  2- Type of metrics to evaluate model performance with
  3- The chosen dataset to train model over

  It initiates three pure functions, one for downloading and balancing the data,
  the second one calls the model and starts training, while the third one saves the model
  and evaluates other statistical metrics.
  '''
  #Decode parameters
  if params:
    dataset = params['dataset']
    logpars = params['log_p'][:4]
    savedir = params['log_p'][-1]

    epochs,batchsize,n_classes  = params['train_p']['epochs'],params['train_p']['batchsize'],params['train_p']['n_classes']

  print(n_classes)
  if not os.path.isdir(savedir):
    os.mkdir(savedir)

  #First, load and balance dataset to make it ready for training
  DATA_X,DATA_Y,TRAIN_I,VAL_I,TEST_I = load_data(dataset = dataset)
  data_stats(DATA_X,DATA_Y,TEST_I,names=['Connection','No-Connection','GeneA->R->GeneB','GeneB->R->GeneA'],savedir=savedir)

  FindrML = FindrML_TRAINER(epochs,batchsize,n_classes,DATA={'data':[DATA_X,DATA_Y],'index':[TRAIN_I,VAL_I,TEST_I]})

  [GetROCcurve,GetPRcurve,GetMCC,SaveModel,SaveDir]

  print('\n- - - - - - - - - - - - - - - - - - - - - -\n')
  print('Metrics Scores:')
  print('- - - - - - - - - - - - - - - - - - - - - -\n')
  if logpars[0]: calc_AUC(True,savedir,'FindrML_ROC',preds=FindrML.predict(DATA_X[TEST_I]),labels=DATA_Y[TEST_I][:,:n_classes],n_classes=n_classes)
  print('-----\n')
  if logpars[1]: calc_PR(preds=FindrML.predict(DATA_X[TEST_I]),labels=DATA_Y[TEST_I][:,:n_classes],n_classes=n_classes,savedir=savedir)
  print('-----\n')
  if logpars[2]: calc_MCC(preds=FindrML.predict(DATA_X[TEST_I]),labels=DATA_Y[TEST_I][:,:n_classes],n_classes=n_classes)
  print('-----\n')
  if logpars[3]: save_model(kerasmodel=FindrML,savedir=savedir,name='FindrML')
  print('- - - - - - - - - - - - - - - - - - - - - -\n')


In [ ]:
def FindrML_TRAINER(*params,**DATA):
  if DATA:
    DATA_X,DATA_Y = DATA['DATA']['data']
    TRAIN_I,VAL_I,TEST_I = DATA['DATA']['index']
  print(len(TRAIN_I))
  print(len(VAL_I))
  print(len(TEST_I))

  if params:
    epochs,batchsize,n_classes = params[0],params[1],params[2]
  else:
    epochs,batchsize,n_classes = 300,8,4

  TRAIN_G = MULTI_G(DATA_X[TRAIN_I],DATA_Y[TRAIN_I][:,:n_classes],batch_size=batchsize,n_classes=n_classes)
  VAL_G   = MULTI_G(DATA_X[VAL_I]  ,DATA_Y[VAL_I][:,:n_classes]  ,batch_size=1 ,n_classes=n_classes)
  TEST_G  = MULTI_G(DATA_X[TEST_I] ,DATA_Y[TEST_I][:,:n_classes] ,batch_size=1 ,n_classes=n_classes)

  FindrML  = Shallow_M(n_classes=n_classes,act='softmax')

  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=2500,
    decay_rate=0.8)
  optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
  FindrML.compile(optimizer=optimizer,loss='CategoricalCrossentropy' ,
                  metrics=['Accuracy',tf.keras.metrics.AUC(multi_label=True)])

  earlystop = tf.keras.callbacks.EarlyStopping('val_loss',patience=100,restore_best_weights=True)
  evaluator = Evaluation(VAL_G, DATA_Y[VAL_I], TEST_G, DATA_Y[TEST_I],multi=True)

  FindrML.fit(TRAIN_G,epochs=epochs,validation_data=VAL_G,callbacks=[earlystop])

  return FindrML

In [ ]:
def get_data(dataset):
  scripts = []
  for dirs,_,fils in os.walk('/content/'):
    for file in fils:
      if '.sh' in file:
        scripts.append(os.path.join(dirs,file))

  if 'yeastract' in dataset.lower():
    script = [i for i in scripts if 'yeast' in i][0]
  elif 'dream'   in dataset.lower():
    script = [i for i in scripts if 'dream' in i][0]

  subprocess.call(['chmod','+777',script])
  subprocess.call(['./'+script[9:]])


def preprocess_yeastract(dirc):
  def open_file(direc):
    file = open(direc,'r+')

    for row in file.readlines():
      row    = row.split('\n')[0]
      elems  = row.split(',')
      yield elems

  class numpy_encoder(json.JSONEncoder):
    def default(self,obj):
      if isinstance(obj,np.ndarray):
        return obj.tolist()
      return json.JSONEncoder.default(self, obj)

  def ret(row,columns):
    keys = [row +'-'+c if i%2 ==0 else c+'-'+row for i,c in enumerate(columns)]
    return keys

  yeastract = {}
  files  = {}

  for dirs,keys,fils in os.walk(dirc):
    for fil in fils:
      if 'expression' in fil.lower():
        files['expression'] = os.path.join(dirs,fil)

      elif 'regulation' in fil.lower():
        files['regulation'] = os.path.join(dirs,fil)

  #To get gene expressions
  genes_file  = open_file(files['expression'])

  genes_N     = [gene for gene in next(genes_file)[1:]]
  expressions = []
  genes       = {}


  for row in genes_file:
    expressions.append(row[1:])

  expressions = np.array(expressions,dtype=np.float64).T

  for j,gene in enumerate(genes_N):
    genes[gene] = expressions[j]

  del expressions
  #To store labels
  groundTs = open_file(files['regulation'])
  columns  = next(groundTs)[1:]
  rows     = []

  matrix   = []
  for p,groundT in enumerate(groundTs):
    rows.append(groundT[0])
    matrix.append(list(map(int,groundT[1:])))

  matrix = np.reshape(np.array(matrix),(len(rows),len(columns)))
  for i,row in enumerate(rows):
    keys = ret(row,columns)
    for j,key in enumerate(keys):
      if key not in yeastract:
        yeastract[key] = {}
        try:
          yeastract[key]['Expression_A'] = genes[key.split('-')[0]]
          yeastract[key]['Expression_B'] = genes[key.split('-')[1]]
        except:
          del yeastract[key]
          continue
        yeastract[key]['Labelx']= matrix[i,j]
        yeastract[key]['Labely']= 1 - matrix[i,j]
        yeastract[key]['P']     = 1 if (matrix[i,j] == 1 and j%2 ==0) else 0
        yeastract[key]['N']     = 1 if (matrix[i,j] == 1 and j%2 !=0) else 0
      else:
        continue

  return yeastract

def preprocess_dream(dirc):
  DATA = None
  for dirs,keys,fils in os.walk(dirc):
    for fil in fils:
      if 'preprocessed.json' in fil.lower():
        JFILE = open(os.path.join(dirs,fil),'r')
        DATA  = json.load(JFILE)

  return DATA


def extract(data,sampler=50000,specify=False):
  E_DATA = {'geneEx':[],'label':[]}

  if specify:
    selector = list(data.keys())[:sampler]
  else:
    selector = list(data.keys())

  for key in selector:
    E_DATA['geneEx'].append([data[key]['Expression_A'],data[key]['Expression_B']])
    label = []
    E_DATA['label'].append([data[key]['Labelx'],data[key]['Labely'],data[key]['P'],data[key]['N']])

  E_DATA['geneEx'] = np.array(E_DATA['geneEx'], dtype=np.float64)
  E_DATA['label']  = np.array(E_DATA['label'])

  return E_DATA

def balance(labely,tobalance_class = 0):
  '''
  Takes:
  1-Labels for two or more multi-label classification problem
  2-Which class to balance over
  Returns: Indices for balanced train-val-test splits
  '''

  one_index = [i for i in range(len(labely)) if labely[i][tobalance_class] == 1]
  zer_index = [i for i in range(len(labely)) if labely[i][tobalance_class] == 0]

  np.random.shuffle(one_index)
  np.random.shuffle(zer_index)

  R_index = [i for j in [one_index , zer_index[:len(one_index)]] for i in j]

  np.random.shuffle(R_index)

  train_index = R_index[:int(0.8*len(R_index))]
  val_index   = R_index[int(0.8*len(R_index)):int(0.9*len(R_index))]
  test_index  = R_index[int(0.9*len(R_index)):]

  return train_index,val_index,test_index

def data_stats(input_X,label_Y,indices,names=None,savedir=None):
  t_d = input_X[indices]
  t_dy= label_Y[indices]

  if names: classes = names
  else: classes = ['class-'+str(i) for i in range(t_dy.shape[1])]

  values  = []
  for i in range(t_dy.shape[1]):
    values.append(len(np.where(t_dy[:,i]==1)[0]))

  if not os.path.isdir(savedir):
    os.mkdir(savedir)

  plt.figure(figsize=(25, 10))
  plt.subplot(131)
  plt.bar(names, values)

  plt.title('Data Statistics')
  plt.savefig(os.path.join(savedir,'Data_Statistics.png'))
  plt.close()

def load_data(dataset):
  DATA = {}

  #Run scripts to load data from cloned repo
  get_data(dataset)
  #Preprocess Data
  if 'yeastract' in dataset.lower():
    DATA = preprocess_yeastract('/content')
  else:
    DATA = preprocess_dream('/content')

  #Extract inputX and labelY from the json file
  DATA   = extract(DATA,specify=True)

  #Balance data labels
  TRAIN_I,VAL_I,TEST_I = balance(DATA['label'],tobalance_class=0)

  return [DATA['geneEx'],DATA['label'],TRAIN_I,VAL_I,TEST_I]


In [ ]:
def save_roc(*scores,name='ROC_CURVE.png',ROCdir='/content',n_classes=3):
  if scores:
    fpr,tpr,roc_auc = scores[0],scores[1],scores[2]
  else:
    print('Please insert scores to plot the roc curve')
    return

  if n_classes >1 and not isinstance(fpr,dict) :
    print('please enter scores for all classes to continue ..')
    return

  if not os.path.isdir(ROCdir):
    os.mkdir(ROCdir)

    # Plot all ROC curves
  plt.figure(figsize=[8, 8])
  lw = 2
  plt.plot(fpr["micro"], tpr["micro"],label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]),
          color='deeppink', linestyle=':', linewidth=4)

  colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
  for i, color in zip(range(n_classes), colors):
      plt.plot(fpr[i], tpr[i], color=color, lw=lw,
              label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

  plt.plot([0, 1], [0, 1], 'k--', lw=lw)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Model Performance')
  plt.legend(loc="lower right")
  plt.savefig(os.path.join(ROCdir,name+'.png'))
  plt.close()

def calc_AUC(*saveargs,preds,labels,n_classes=4):
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(labels[:, i], preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

  fpr["micro"], tpr["micro"], _ = roc_curve(labels.ravel(), preds.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  print('Micro-Average = {}\n'.format(roc_auc["micro"]))
  if saveargs:
    print('Saving ROC curve ...\n')
    save,path,name = saveargs[0],saveargs[1],saveargs[2]
    if save:
      save_roc(fpr,tpr,roc_auc,name=name,ROCdir=path,n_classes=n_classes)
      print('ROC curve is saved ..\n')

def save_model(kerasmodel,savedir,name):

  if '.' in name:
    name = ''.join(name.split('.')[0])

  if not os.path.isdir(savedir):
    os.mkdir(savedir)

  kerasmodel.save(os.path.join(savedir,name),save_format='tf')
  print('Model is saved ..\n')

def calc_MCC(preds,labels,n_classes):
  print('Matthews Correlation Coefficients:\n')
  for i in range(n_classes):
    print('Class {}: {}\n'.format(i,matthews_corrcoef(labels[:, i], preds[:, i].round())))
  print('\n')

def calc_PR(preds,labels,n_classes,savedir):
  # For each class
  precision = dict()
  recall = dict()
  average_precision = dict()

  for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(labels[:, i],preds[:, i])

  # A "micro-average": quantifying score on all classes jointly
  precision["micro"], recall["micro"], _ = precision_recall_curve(labels.ravel(),preds.ravel())

  average_precision["micro"] = average_precision_score(labels, preds,average="micro")

  plt.figure(figsize=[10, 10])
  lw = 2
  plt.step(recall['micro'], precision['micro'], where='post')

  plt.xlabel('Recall')
  plt.ylabel('Precision')
  plt.ylim([0.0, 1.05])
  plt.xlim([0.0, 1.0])
  plt.title('Average precision score, micro-averaged over all classes: AP={0:0.2f}'.format(average_precision["micro"]))
  plt.savefig(os.path.join(savedir,'PR'+'.png'))
  plt.close()

  print('Precision recall curve is saved ..\n')


In [ ]:

class Evaluation(keras.callbacks.Callback):
  def __init__(self, val_data_gen, val_labels, test_data_gen, test_labels,multi=True):
    super(keras.callbacks.Callback, self).__init__()
    self.test_data   = test_data_gen
    self.val_labels  = val_labels
    self.val_data    = val_data_gen
    self.test_labels = test_labels

    if multi == True:
      self.param = 'ovr'
    else:
      self.param = 'raise'

  def on_epoch_end(self, epoch, logs=None):
    y_preds = self.model.predict_generator(self.val_data)
    print(' | val_auc:', roc_auc_score(self.val_labels[:len(y_preds)], y_preds,multi_class=self.param))

    y_preds = self.model.predict_generator(self.test_data)
    print(' | test_auc:', roc_auc_score(self.test_labels[:len(y_preds)], y_preds,multi_class=self.param))

class MULTI_G(tf.keras.utils.Sequence):
  def __init__(self,genes,labels,batch_size,n_classes=1,shuffle=True):
    self.batch_size = batch_size
    self.labels     = labels
    self.genes      = genes
    self.n_classes  = n_classes
    self.shuffle    = shuffle
    self.dim = self.genes.shape[1:]
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(len(self.genes) / self.batch_size))

  def __getitem__(self, index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    X, y = self.__data_generation(indexes)
    return X, y

  def on_epoch_end(self):
    self.indexes = np.arange(len(self.genes))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, *self.dim))
    y = np.empty((self.batch_size,self.n_classes))

    # Generate data
    for i, ID in enumerate(list_IDs_temp):
      #Melspec
      X[i,]  = self.genes[ID]

      # Store class
      y[i,]  = self.labels[ID]

    return X, y

class Shallow_M (tf.keras.Model):
  def __init__(self,n_classes=1,act='sig',training=True):
    super(Shallow_M,self).__init__()
    self.DenseA   = tf.keras.layers.Dense(256,activation=tf.nn.relu)
    self.BatchN_A = tf.keras.layers.BatchNormalization(momentum=0.9)
    self.DropO_A  = tf.keras.layers.Dropout(rate=0.5)

    self.DenseB   = tf.keras.layers.Dense(256,activation=tf.nn.relu)
    self.BatchN_B = tf.keras.layers.BatchNormalization(momentum=0.9)
    self.DropO_B  = tf.keras.layers.Dropout(rate=0.5)
    self.FlattenX = tf.keras.layers.Flatten()

    self.CnnD     = tf.keras.layers.Conv2D(filters=16,kernel_size=2,)
    self.BatchN_D = tf.keras.layers.BatchNormalization(momentum=0.9)
    self.DropO_D  = tf.keras.layers.Dropout(rate=0.5)

    self.CnnE     = tf.keras.layers.Conv2D(filters=8,kernel_size=1,)
    self.BatchN_E = tf.keras.layers.BatchNormalization(momentum=0.9)
    self.DropO_E  = tf.keras.layers.Dropout(rate=0.5)

    self.Flatten  = tf.keras.layers.Flatten()
    self.DenseD   = tf.keras.layers.Dense(32,activation=tf.nn.relu)
    self.BatchN_DD= tf.keras.layers.BatchNormalization(momentum=0.9)
    self.DropO_DD = tf.keras.layers.Dropout(rate=0.5)

    if (act == 'sig'):
      self.Predict  = tf.keras.layers.Dense(n_classes,activation=tf.nn.sigmoid)
    else:
      self.Predict  = tf.keras.layers.Dense(n_classes,activation=tf.nn.softmax)

    self.training = training

  def call(self,inputs):
    x = tf.math.reduce_prod(inputs,axis=1)

    x = self.DenseA(x)
    x = self.BatchN_A(x,)
    x = self.DropO_A(x,training=self.training)

    x = self.DenseB(x)
    x = self.BatchN_B(x,)
    x = self.DropO_B(x,training=self.training)
    x = self.FlattenX(x)

    d = self.CnnD(inputs[:,:,:,np.newaxis])
    d = self.BatchN_D(d)
    d = self.DropO_D(d)

    d = self.CnnE(d)
    d = self.BatchN_E(d)
    d = self.DropO_E(d)

    out = self.Flatten(d)
    out = self.DenseD(out)
    out = self.BatchN_DD(out)
    out = self.DropO_DD(out)

    out = tf.keras.layers.concatenate([x,out])

    return self.Predict(out)
